# Ansible Inventory

Ansible needs to know which hosts it manages. For this, it uses an *inventory*.

## Static inventory

A static inventory is just a text file (by default, `/etc/ansible/hosts`, but we'll change that). 

The inventory lists the hosts that Ansible will manage, optionally collecting them into groups.
One example is in the class repository as `inventory`:

```
[web]
arborian-01.class.arborian.com
arborian-02.class.arborian.com

[db]
arborian-03.class.arborian.com
arborian-04.class.arborian.com

[app]
arborian-05.class.arborian.com
arborian-06.class.arborian.com
```

# Inventory: Dynamic Inventory

We can also use Python modules to create our inventory programmatically (as when we might want to enumerate all hosts on AWS, Azure, or Google Compute Engine)

Any inventory file that is executable will be run to generate an inventory JSON result when fed certain command-line arguments.

Some useful links:

- [Dynamic Inventory](http://docs.ansible.com/ansible/intro_dynamic_inventory.html)
- [Developing your own inventory script](http://docs.ansible.com/ansible/dev_guide/developing_inventory.html)

# Hosts and groups

Each machine we manage is a _host_. 
One or more machines can be included in a _group_.

Example:

```
[web]
arborian-01.class.arborian.com
arborian-02.class.arborian.com

[db]
arborian-03.class.arborian.com
arborian-04.class.arborian.com

[app]
arborian-05.class.arborian.com
arborian-06.class.arborian.com

[webapp:children]
web
app
```

# Host and group Variables

You can define variables for each host and group. 
Variables can be used in `playbooks` and `roles` (which we'll get to later).

To define a host variable, just add it to the line where the host is defined:

```
[web]
arborian-01.class.arborian.com maxRequestsPerChild=1000
arborian-02.class.arborian.com maxRequestsPerChild=1080
```

To define variables which apply to all members of a group, we use the `[GROUP:vars]` syntax:

```
[web:vars]
http_port=80
datacenter=atl
```

# Variables and groups in Vagrant

We can set up our groups and variables in vagrant by using the `ansible.groups` command in our Vagrantfile:

```ruby
  config.vm.provision "ansible" do |ansible|
    ansible.playbook = "playbooks/basic.yaml"
    ansible.groups = {
      "webdb" => ["web", "db"],
      "webdb:vars" => {"ntp_server" => "ntp.atlanta.example.com",
                       "proxy" => "proxy.atlanta.example.com"}
    }
  end

```


# Checking the value of a variable

To check the value of a variable, we can use Ansible's `debug` command:

In [3]:
cd ..

/Users/rick446/src/Classes/Ansible


In [4]:
%%bash
ansible web -m debug -a 'var=proxy'

web | SUCCESS => {
    "proxy": "proxy.atlanta.example.com"
}


# Ad-hoc Commands

Ansible command line

`ansible <host pattern> [options]`

Frequently used options

- `-m MODULE_NAME` (default is command)
- `-a MODULE_ARGS`

Frequently used host patterns

- Specific hostname
- `localhost`
- `all`
- `GROUP-NAME` (one of the groupings from your inventory)

# Patterns 

In simple situations, our pattern will be either a host or a group name. 
Ansible, however, gives us [quite a bit of flexibility][patterns] in the patterns we use.

For instance, you can:

- combine hosts and groups `group:group2`
- use "globbing" expressions `web-*`
- exclude hosts `web-*:!web-01`
- require membership in a group `web:db:&atlanta` (all `web` OR `db` hosts that are also in the `atlanta` group)
- use regular expressions `~(web|db).*\.example\.com`

[patterns]: http://docs.ansible.com/ansible/latest/intro_patterns.html



# Lab

## Update some of the groups and variables in the Vagrant ansible config (in the class directory)

(To apply changes to the `Vagrantfile`, the following command is handy:)

```vagrant reload --provision```

Check your updates using `vagrant all -m debug var=VARNAME`

## Use different patterns to target different sets of your infrastructure. 

Try using the following command to see which hosts respond:

```ansible PATTERN -m ping```

# Lab - for Windows users

Update the `inventory` file to create/modify some groups and variables

Check your updates using `vagrant all -m debug var=VARNAME`

## Use different patterns to target different sets of your infrastructure. 

Try using the following command to see which hosts respond:

```ansible PATTERN -m ping```